In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

### 合并数据集

#### 1. 数据风格的DataFrame合并

In [2]:
df1 = DataFrame({"key":["b", "b", "a", "c", "a", "a", "b"],
                "data1": range(7)})

In [3]:
df2 = DataFrame({"key": ["a", "b", "d"],
                "data2": range(3)})

In [4]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [5]:
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [6]:
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [7]:
pd.merge(df1, df2, on="key")

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [8]:
df3 = DataFrame({"lkey":["b", "b", "a", "c", "a", "a", "b"],
                "data1": range(7)})

In [9]:
df4 = DataFrame({"rkey": ["a", "b", "d"],
                "data2": range(3)})

In [10]:
# 默认情况，merge做的是“inner”连接，结果中的键是交集
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [11]:
# 外连接求取的键的并集
pd.merge(df1, df2, how="outer")

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [12]:
# 多对多的合并操作
df1 = DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                "data1":range(6)})
df2 = DataFrame({"key": ["a", "b", "a", "b", "d"],
                "data2": range(5)})

In [13]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [14]:
df2

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


In [15]:
# 多对多连接产生的是行的笛卡尔积
pd.merge(df1, df2, on="key", how="left")

,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


In [16]:
pd.merge(df1, df2, how="inner")

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


In [17]:
# 根据多个键进行合并
left = DataFrame({"key1": ["foo", "foo", "bar"],
                 "key2": ["one", "two", "one"],
                 "lval": [1, 2, 3]})

In [18]:
right = DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                  "key2": ["one", "one", "one", "two"],
                  "rval": [4, 5, 6, 7]})

In [19]:
pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [20]:
# 对于合并运算需要考虑的最后一个问题是对重复列名的处理
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [21]:
pd.merge(left, right, on="key1", suffixes=["_left", "_right"])

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


#### 2.索引上的合并

In [22]:
left1 = DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                  "value": range(6)})

In [23]:
right1 = DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])

In [24]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [25]:
right1

,group_val
a,3.5
b,7.0


In [26]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [27]:
# 通过外链接的方式得到并集
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [28]:
# 处理层次化索引的数据
lefth = DataFrame({"key1": ["Ohio", "Ohio", "Ohio", "Neveda", "Neveda"],
                  "key2": [2000, 2001, 2002, 2001, 2002],
                  "data": range(5)})

In [29]:
righth = DataFrame(np.arange(12).reshape((6, 2)),
                  index=[["Neveda", "Neveda", "Ohio", "Ohio", "Ohio", "Ohio"],
                        [2001, 2000, 2000, 2000, 2001, 2002]],
                  columns=["event1", "event2"])

In [30]:
lefth

,data,key1,key2
0,0,Ohio,2000
1,1,Ohio,2001
2,2,Ohio,2002
3,3,Neveda,2001
4,4,Neveda,2002


In [31]:
righth

event1  event2
Neveda 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [32]:
# 以列表的形式指明用作合并键的多个列（注意对重复索引值的处理）
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)

,data,key1,key2,event1,event2
0,0,Ohio,2000,4,5
0,0,Ohio,2000,6,7
1,1,Ohio,2001,8,9
2,2,Ohio,2002,10,11
3,3,Neveda,2001,0,1


In [33]:
pd.merge(lefth, righth, left_on=["key1", "key2"],
        right_index=True, how="outer")

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4.0,5.0
0,0.0,Ohio,2000,6.0,7.0
1,1.0,Ohio,2001,8.0,9.0
2,2.0,Ohio,2002,10.0,11.0
3,3.0,Neveda,2001,0.0,1.0
4,4.0,Neveda,2002,NaN,NaN
4,NaN,Neveda,2000,2.0,3.0


In [34]:
# 同时使用双方的索引
left2 = DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=["a", "c", "e"],
                 columns=["Ohio", "Neveda"])

In [35]:
right2 = DataFrame([[7., 8.], [9., 10.], [11., 12.], [13., 14.]],
                  index=["b", "c", "d", "e"], columns=["Missouri", "Alabama"])

In [36]:
left2

,Ohio,Neveda
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [37]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [38]:
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Ohio,Neveda,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [39]:
left2.join(right2, how="outer")

,Ohio,Neveda,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [40]:
left1.join(right1, on="key")

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [41]:
# 简单的索引合并
another = DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                   index=["a", "c", "e", "f"], columns=["New York", "Oregon"])

In [42]:
left2.join([right2, another])

,Ohio,Neveda,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [43]:
left2.join([right2, another], how="outer", sort=True)

/SoftWare/python/python3-data-analysis-F15e879i/lib/python3.5/site-packages/pandas/core/frame.py:6359: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  verify_integrity=True)


,Ohio,Neveda,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


#### 3.轴向连接

In [44]:
arr = np.arange(12).reshape((3, 4))

In [45]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [46]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [47]:
# 使用pandas的concat函数
s1 = Series([0, 1], index=["a", "b"])
s2 = Series([2, 3, 4], index=["c", "d", "e"])
s3 = Series([5, 6], index=["f", "g"])

In [48]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [49]:
# 默认情况下，concat是在axis=0上工作的，最终产生一个新的Series。
# 如果传入axis=1，则结果就会变成一个DataFrame（axis=1是列）。

In [50]:
pd.concat([s1, s2, s3], axis=1)

/SoftWare/python/python3-data-analysis-F15e879i/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [51]:
s4 = pd.concat([s1 * 5, s3])

In [52]:
pd.concat([s1, s4], axis=1)

/SoftWare/python/python3-data-analysis-F15e879i/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


In [53]:
pd.concat([s1, s4], axis=1, join="inner")

,0,1
a,0,0
b,1,5


In [54]:
pd.concat([s1, s4], axis=1, join_axes=[["a", "c", "b", "e"]])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,5.0
e,NaN,NaN


In [55]:
result = pd.concat([s1, s2, s3], keys=["one", "two", "three"])

In [56]:
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [57]:
result.unstack()

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


In [58]:
pd.concat([s1, s2, s3], axis=1, keys=["one", "two", "three"])

/SoftWare/python/python3-data-analysis-F15e879i/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [59]:
# 同样的逻辑对DataFrame对象也是一样
df1 = DataFrame(np.arange(6).reshape(3, 2), index=["a", "b", "c"],
               columns=["one", "two"])
df2 = DataFrame(5 + np.arange(4).reshape(2, 2), index=["a", "c"],
               columns=["three", "four"])

In [60]:
pd.concat([df1, df2], axis=1, keys=["level1", "level2"])

/SoftWare/python/python3-data-analysis-F15e879i/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [61]:
pd.concat({"level1": df1, "level2": df2}, axis=1)

/SoftWare/python/python3-data-analysis-F15e879i/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [62]:
# 管理层次化索引创建方式
pd.concat([df1, df2], axis=1, keys=["level1", "level2"],
         names=["uper", "lower"])

/SoftWare/python/python3-data-analysis-F15e879i/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  This is separate from the ipykernel package so we can avoid doing imports until


uper  level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [63]:
df1 = DataFrame(np.random.randn(3, 4), columns=["a", "b", "c", "d"])
df2 = DataFrame(np.random.randn(2, 3), columns=["b", "d", "a"])

In [64]:
df1

,a,b,c,d
0,-1.058364,-0.959214,-1.282154,-1.261860
1,-0.889509,-0.602209,-0.267856,-0.357238
2,-0.511590,0.435446,0.257792,-0.107340


In [65]:
df2

,b,d,a
0,0.502249,1.312790,-0.037103
1,-0.349934,3.482746,0.620700


In [66]:
pd.concat([df1, df2], ignore_index=True)

/SoftWare/python/python3-data-analysis-F15e879i/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


,a,b,c,d
0,-1.058364,-0.959214,-1.282154,-1.261860
1,-0.889509,-0.602209,-0.267856,-0.357238
2,-0.511590,0.435446,0.257792,-0.107340
3,-0.037103,0.502249,NaN,1.312790
4,0.620700,-0.349934,NaN,3.482746


#### 4.合并重叠数据

In [67]:
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
          index=["f", "e", "d", "c", "b", "a"])
b = Series(np.arange(len(a), dtype=np.float64),
          index=["f", "e", "d", "c", "b", "a"])

In [68]:
b[-1] = np.nan

In [69]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [70]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [71]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [72]:
# 用参数对象中的数据为调用者对象的缺失数据“打补丁”
df1 = DataFrame({"a": [1., np.nan, 5., np.nan],
                "b": [np.nan, 2., np.nan, 6.],
                "c": range(2, 18, 4)})
df2 = DataFrame({"a": [5., 4, np.nan, 3., 7.],
                "b": [np.nan, 3., 4., 6., 8.]})

In [73]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


### 重塑和轴向旋转

#### 1.重塑层次化索引

In [74]:
data = DataFrame(np.arange(6).reshape((2, 3)),
                index=pd.Index(["Ohio", "Colorado"], name="state"),
                columns=pd.Index(["one", "two", "three"], name="number"))

In [75]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [76]:
# 使用stack将数据的列“旋转”为行
result = data.stack()

In [77]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [78]:
# 使用unstack将数据的行“旋转”为列
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [79]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [80]:
result.unstack("state")

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [81]:
# 如果不是所有的级别值都能在各个分组中找到的话，unstack操作可能会引入缺失数据
s1 = Series([0, 1, 2, 3], index=["a", "b", "c", "d"])
s2 = Series([4, 5, 6], index=["c", "d", "e"])
data2 = pd.concat([s1, s2], keys=["one", "two"])

In [82]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [83]:
# stack默认会过滤除缺失数据，该运算是可逆的
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [84]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [85]:
# 在对DataFrame进行unstack操作时，作为旋转轴的级别将会成为结果中的最低级别
df = DataFrame({"left": result, "right": result + 5},
              columns=pd.Index(["left", "right"], name="side"))

In [86]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [87]:
df.unstack("state")

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [88]:
df.unstack("state").stack("side")

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### 数据库转换

#### 1.移除重复数据

In [89]:
data = DataFrame({"k1":["one"] * 3 + ["two"] * 4,
                 "k2":[1, 1, 2, 3, 3, 4, 4]})

In [90]:
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [91]:
# 表示各行是否是重复行
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [92]:
# 移除重复行的DataFrame
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [93]:
# 只根据k1列过滤重复项
data["v1"] = range(7)

In [94]:
data.drop_duplicates(["k1"])

,k1,k2,v1
0,one,1,0
3,two,3,3


#### 2.利用函数或映射进行数据转换

In [95]:
data = DataFrame({"food":["bacon", "pulled port", "bacon", "Pastrami", 
                         "corned beef", "Bacon", "pastrami", "honey ham",
                         "nova lox"],
                 "ounces":[4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

In [96]:
data

,food,ounces
0,bacon,4.0
1,pulled port,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [97]:
# 肉类动物的映射
meat_to_animal = {
    "bacon": "pig",
    "pulled pork": "pig",
    "pastrami": "cow",
    "honey ham": "pig",
    "nova lox": "salmon"
}

In [98]:
# 将各个值转换为小写
# 使用map是一种实现元素转换以及其他数据清理工作的便捷方式
data["animal"] = data["food"].map(str.lower).map(meat_to_animal)

In [99]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled port,3.0,NaN
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,NaN
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


#### 3.替换值

In [100]:
data = Series([1., -999., 2., -999., -1000, 3.])

In [101]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [102]:
# 替换值
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [103]:
# 替换多个值
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [104]:
# 对不同的值进行不同的替换
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [105]:
# 传入参数可以是字典
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

#### 4.重命名轴索引

In [106]:
data = DataFrame(np.arange(12).reshape((3, 4)),
                index=["Ohio", "Colorado", "New York"],
                columns=["one", "two", "three", "four"])

In [107]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [108]:
data.index.map(str.lower)

Index(['ohio', 'colorado', 'new york'], dtype='object')

In [109]:
# 修改index
data.index = data.index.map(str.lower)

In [110]:
data

,one,two,three,four
ohio,0,1,2,3
colorado,4,5,6,7
new york,8,9,10,11


In [111]:
# 创建数据集的转换版（而不是修改原始数据）
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [112]:
# rename可以结合字典型对象实现对部分轴标签的更新
# rename：复制DataFrame并对其索引和列标签进行赋值
data.rename(index={"ohio":"INDIANA"},
           columns={"three":"peekaboo"})

,one,two,peekaboo,four
INDIANA,0,1,2,3
colorado,4,5,6,7
new york,8,9,10,11


In [113]:
# 就地修改某个数据集
# 总是返回DataFrame的引用
_ = data.rename(index={"ohio":"INDIANA"}, inplace=True)

In [114]:
data

,one,two,three,four
INDIANA,0,1,2,3
colorado,4,5,6,7
new york,8,9,10,11


#### 5.离散化和面元划分

In [115]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [116]:
# 将数据划分为“18到25”、“26到35”、“35到60”以及“60以上”
bins = [18, 25, 35, 60, 100]

In [117]:
cats = pd.cut(ages, bins)

In [118]:
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [119]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

In [120]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [121]:
# 设置闭端
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [122]:
group_names = ["Youth", "YoungAdult", "MiddleAged", "Senior"]

In [123]:
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [124]:
data = np.random.rand(20)

In [125]:
pd.cut(data, 4, precision=2)

[(0.42, 0.58], (0.26, 0.42], (0.58, 0.75], (0.58, 0.75], (0.58, 0.75], ..., (0.091, 0.26], (0.42, 0.58], (0.42, 0.58], (0.091, 0.26], (0.26, 0.42]]
Length: 20
Categories (4, interval[float64]): [(0.091, 0.26] < (0.26, 0.42] < (0.42, 0.58] < (0.58, 0.75]]

In [126]:
# qcut是一个非常类似于cut的函数，它可以根据样本分位数对数据进行面元划分。
# 正态分布
data = np.random.randn(1000)

In [127]:
# 按四分位数进行切割
cats = pd.qcut(data, 4)

In [128]:
cats

[(-0.659, -0.000659], (-0.659, -0.000659], (0.697, 3.29], (-0.659, -0.000659], (-0.659, -0.000659], ..., (-0.659, -0.000659], (-3.201, -0.659], (-3.201, -0.659], (0.697, 3.29], (-0.000659, 0.697]]
Length: 1000
Categories (4, interval[float64]): [(-3.201, -0.659] < (-0.659, -0.000659] < (-0.000659, 0.697] < (0.697, 3.29]]

In [129]:
pd.value_counts(cats)

(0.697, 3.29]          250
(-0.000659, 0.697]     250
(-0.659, -0.000659]    250
(-3.201, -0.659]       250
dtype: int64

In [130]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-1.29, -0.000659], (-1.29, -0.000659], (1.314, 3.29], (-1.29, -0.000659], (-1.29, -0.000659], ..., (-1.29, -0.000659], (-1.29, -0.000659], (-3.201, -1.29], (-0.000659, 1.314], (-0.000659, 1.314]]
Length: 1000
Categories (4, interval[float64]): [(-3.201, -1.29] < (-1.29, -0.000659] < (-0.000659, 1.314] < (1.314, 3.29]]

#### 6.检测和过滤异常值

In [131]:
np.random.seed(12345)

In [132]:
data = DataFrame(np.random.randn(1000, 4))

In [133]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


In [134]:
data

,0,1,2,3
0,-0.204708,0.478943,-0.519439,-0.555730
1,1.965781,1.393406,0.092908,0.281746
2,0.769023,1.246435,1.007189,-1.296221
3,0.274992,0.228913,1.352917,0.886429
4,-2.001637,-0.371843,1.669025,-0.438570
5,-0.539741,0.476985,3.248944,-1.021228
6,-0.577087,0.124121,0.302614,0.523772
7,0.000940,1.343810,-0.713544,-0.831154
8,-2.370232,-1.860761,-0.860757,0.560145
9,-1.265934,0.119827,-1.063512,0.332883


In [135]:
col = data[3]

In [136]:
col[np.abs(col) > 3]

97     3.927528
305   -3.399312
400   -3.745356
Name: 3, dtype: float64

In [137]:
# 选出全部含有“超过3或-3的值”
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


In [138]:
# 将值限制在区间-3到3以内
data[np.abs(data) > 3] = np.sign(data) * 3

In [139]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067623,0.068473,0.025153,-0.002081
std,0.995485,0.990253,1.003977,0.989736
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.000000,2.653656,3.000000,3.000000


#### 7.排列和随机采样

In [140]:
df = DataFrame(np.arange(5 * 4).reshape(5, 4))

In [141]:
sampler = np.random.permutation(5)

In [142]:
sampler

array([1, 0, 2, 3, 4])

In [143]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [144]:
df.take(sampler)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [145]:
# 从permutation返回的数组中切下前k个元素，其中k为期望的子集大小。
df.take(np.random.permutation(len(df))[:3])

,0,1,2,3
1,4,5,6,7
3,12,13,14,15
4,16,17,18,19


In [146]:
bag = np.array([5, 7, -1, 6, 4])

In [147]:
sampler = np.random.randint(0, len(bag), size=10)

In [148]:
sampler

array([4, 4, 2, 2, 2, 0, 3, 0, 4, 1])

In [149]:
draws = bag.take(sampler)

In [150]:
draws

array([ 4,  4, -1, -1, -1,  5,  6,  5,  4,  7])

#### 8.计算指标/哑变量

In [151]:
df = DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
               "data1": range(6)})

In [152]:
df

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [153]:
pd.get_dummies(df["key"])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [154]:
# 给指标DataFrame的列加上一个前缀，以便能够跟其他数据进行合并。
# get_dummies的prefix参数可以实现该功能。
dummies = pd.get_dummies(df["key"], prefix="key")

In [155]:
df_with_dummy = df[["data1"]].join(dummies)

In [156]:
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [157]:
# 一个对统计应用有用的秘诀是：结合get_dummies和诸如cut之类的离散化函数。
values = np.random.rand(10)

In [158]:
values

array([0.75603383, 0.90830844, 0.96588737, 0.17373658, 0.87592824,
       0.75415641, 0.163486  , 0.23784062, 0.85564381, 0.58743194])

In [159]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [160]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,1,0
1,0,0,0,0,1
2,0,0,0,0,1
3,1,0,0,0,0
4,0,0,0,0,1
5,0,0,0,1,0
6,1,0,0,0,0
7,0,1,0,0,0
8,0,0,0,0,1
9,0,0,1,0,0


### 字符串操作

#### 1.字符串对象方法

In [161]:
content = "life is short, you need python"

In [162]:
content.split(",")

['life is short', ' you need python']

In [163]:
# 修剪空白符
pieces = [x.strip() for x in content.split(",")]

In [164]:
pieces

['life is short', 'you need python']

In [165]:
first, second = pieces

In [166]:
first

'life is short'

In [167]:
second

'you need python'

In [168]:
# 连接字符串
"$".join(pieces)

'life is short$you need python'

In [169]:
# 子字符串定位
"life" in content

True

In [170]:
content.index(",")

13

In [171]:
content.find("you")

15

In [172]:
content.index("you")

15

In [173]:
content.count(",")

1

In [174]:
content.replace(",", "$$$")

'life is short$$$ you need python'

#### 2.正则表达式

In [175]:
import re

In [176]:
content = "life is short, you need python, 168"

In [177]:
re.split("\s+", content)

['life', 'is', 'short,', 'you', 'need', 'python,', '168']

In [178]:
# 打算对许多字符串应用同一正则表达式，强烈建议通过re.compile创建regex对象（可以节省大量的CPU时间）
regex = re.compile("\s+")

In [179]:
regex.split(content)

['life', 'is', 'short,', 'you', 'need', 'python,', '168']

In [180]:
regex.findall(content)

[' ', ' ', ' ', ' ', ' ', ' ']

#### 3.pandas中矢量化的字符串函数

In [181]:
data = {"Dave": "dave@gmail.com", "Kevin": "kevin@gmail.com", "Rob": np.nan}

In [182]:
data = Series(data)

In [183]:
data

Dave      dave@gmail.com
Kevin    kevin@gmail.com
Rob                  NaN
dtype: object

In [184]:
data.isnull()

Dave     False
Kevin    False
Rob       True
dtype: bool

In [185]:
data.str.contains("gmail")

Dave     True
Kevin    True
Rob       NaN
dtype: object

In [186]:
pattern = "([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2, 4})"

In [187]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2, 4})'

In [188]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave      []
Kevin     []
Rob      NaN
dtype: object

### 示例：USDA食品数据库

In [189]:
import json

In [190]:
db = json.load(open("datasets/usda_food/database.json"))

In [191]:
len(db)

6636

In [192]:
db[0].keys()

dict_keys(['nutrients', 'description', 'tags', 'manufacturer', 'portions', 'id', 'group'])

In [193]:
db[0]["nutrients"][0]

{'description': 'Protein',
 'group': 'Composition',
 'units': 'g',
 'value': 25.18}

In [194]:
nutrients = DataFrame(db[0]["nutrients"])

In [195]:
nutrients[:7]

,description,group,units,value
0,Protein,Composition,g,25.18
1,Total lipid (fat),Composition,g,29.20
2,"Carbohydrate, by difference",Composition,g,3.06
3,Ash,Other,g,3.28
4,Energy,Energy,kcal,376.00
5,Water,Composition,g,39.28
6,Energy,Energy,kJ,1573.00


In [196]:
# 取出食物的名称、分类、编号以及制造商
info_keys = ["description", "group", "id", "manufacturer"]

In [197]:
info = DataFrame(db, columns=info_keys)

In [198]:
info[:5]

,description,group,id,manufacturer
0,"Cheese, caraway",Dairy and Egg Products,1008,
1,"Cheese, cheddar",Dairy and Egg Products,1009,
2,"Cheese, edam",Dairy and Egg Products,1018,
3,"Cheese, feta",Dairy and Egg Products,1019,
4,"Cheese, mozzarella, part skim milk",Dairy and Egg Products,1028,


In [199]:
info

,description,group,id,manufacturer
0,"Cheese, caraway",Dairy and Egg Products,1008,
1,"Cheese, cheddar",Dairy and Egg Products,1009,
2,"Cheese, edam",Dairy and Egg Products,1018,
3,"Cheese, feta",Dairy and Egg Products,1019,
4,"Cheese, mozzarella, part skim milk",Dairy and Egg Products,1028,
5,"Cheese, mozzarella, part skim milk, low moisture",Dairy and Egg Products,1029,
6,"Cheese, romano",Dairy and Egg Products,1038,
7,"Cheese, roquefort",Dairy and Egg Products,1039,
8,"Cheese spread, pasteurized process, american, ...",Dairy and Egg Products,1048,
9,"Cream, fluid, half and half",Dairy and Egg Products,1049,


In [200]:
# 查看食物类别的分布情况
pd.value_counts(info.group)[:10]

Vegetables and Vegetable Products    812
Beef Products                        618
Baked Products                       496
Breakfast Cereals                    403
Fast Foods                           365
Legumes and Legume Products          365
Lamb, Veal, and Game Products        345
Sweets                               341
Fruits and Fruit Juices              328
Pork Products                        328
Name: group, dtype: int64

In [201]:
# 对全部营养数据做一些分析，最简单的办法是将所有食物的营养成分整合到一个大表中。
# 实现的步骤:
#    首先，将各食物的营养成分列表转换为一个DataFrame，并添加一个表示编号的列，
#    然后将该DataFrame添加到一个列表中。
#    最后通过concat将这些东西连接起来即可。
nutrients = []

for rec in db:
    fnuts = DataFrame(rec["nutrients"])
    fnuts["id"] = rec["id"]
    nutrients.append(fnuts)
    
nutrients = pd.concat(nutrients, ignore_index=True)

In [202]:
nutrients

,description,group,units,value,id
0,Protein,Composition,g,25.180,1008
1,Total lipid (fat),Composition,g,29.200,1008
2,"Carbohydrate, by difference",Composition,g,3.060,1008
3,Ash,Other,g,3.280,1008
4,Energy,Energy,kcal,376.000,1008
5,Water,Composition,g,39.280,1008
6,Energy,Energy,kJ,1573.000,1008
7,"Fiber, total dietary",Composition,g,0.000,1008
8,"Calcium, Ca",Elements,mg,673.000,1008
9,"Iron, Fe",Elements,mg,0.640,1008


In [203]:
# 丢弃重复项
nutrients.duplicated().sum()

14179

In [204]:
nutrients = nutrients.drop_duplicates()

In [205]:
nutrients

,description,group,units,value,id
0,Protein,Composition,g,25.180,1008
1,Total lipid (fat),Composition,g,29.200,1008
2,"Carbohydrate, by difference",Composition,g,3.060,1008
3,Ash,Other,g,3.280,1008
4,Energy,Energy,kcal,376.000,1008
5,Water,Composition,g,39.280,1008
6,Energy,Energy,kJ,1573.000,1008
7,"Fiber, total dietary",Composition,g,0.000,1008
8,"Calcium, Ca",Elements,mg,673.000,1008
9,"Iron, Fe",Elements,mg,0.640,1008


In [206]:
# 重命名
col_mapping = {"description": "food", "group": "fgroup"}

In [207]:
info = info.rename(columns=col_mapping, copy=False)

In [208]:
info

,food,fgroup,id,manufacturer
0,"Cheese, caraway",Dairy and Egg Products,1008,
1,"Cheese, cheddar",Dairy and Egg Products,1009,
2,"Cheese, edam",Dairy and Egg Products,1018,
3,"Cheese, feta",Dairy and Egg Products,1019,
4,"Cheese, mozzarella, part skim milk",Dairy and Egg Products,1028,
5,"Cheese, mozzarella, part skim milk, low moisture",Dairy and Egg Products,1029,
6,"Cheese, romano",Dairy and Egg Products,1038,
7,"Cheese, roquefort",Dairy and Egg Products,1039,
8,"Cheese spread, pasteurized process, american, ...",Dairy and Egg Products,1048,
9,"Cream, fluid, half and half",Dairy and Egg Products,1049,


In [209]:
col_mapping = {"description": "nutrient", "group": "nutgroup"}

In [210]:
nutrients = nutrients.rename(columns=col_mapping, copy=False)

In [211]:
nutrients

,nutrient,nutgroup,units,value,id
0,Protein,Composition,g,25.180,1008
1,Total lipid (fat),Composition,g,29.200,1008
2,"Carbohydrate, by difference",Composition,g,3.060,1008
3,Ash,Other,g,3.280,1008
4,Energy,Energy,kcal,376.000,1008
5,Water,Composition,g,39.280,1008
6,Energy,Energy,kJ,1573.000,1008
7,"Fiber, total dietary",Composition,g,0.000,1008
8,"Calcium, Ca",Elements,mg,673.000,1008
9,"Iron, Fe",Elements,mg,0.640,1008


In [212]:
# 将info跟nutrients合并起来
ndata = pd.merge(nutrients, info, on="id", how="outer")

In [213]:
ndata

,nutrient,nutgroup,units,value,id,food,fgroup,manufacturer
0,Protein,Composition,g,25.180,1008,"Cheese, caraway",Dairy and Egg Products,
1,Total lipid (fat),Composition,g,29.200,1008,"Cheese, caraway",Dairy and Egg Products,
2,"Carbohydrate, by difference",Composition,g,3.060,1008,"Cheese, caraway",Dairy and Egg Products,
3,Ash,Other,g,3.280,1008,"Cheese, caraway",Dairy and Egg Products,
4,Energy,Energy,kcal,376.000,1008,"Cheese, caraway",Dairy and Egg Products,
5,Water,Composition,g,39.280,1008,"Cheese, caraway",Dairy and Egg Products,
6,Energy,Energy,kJ,1573.000,1008,"Cheese, caraway",Dairy and Egg Products,
7,"Fiber, total dietary",Composition,g,0.000,1008,"Cheese, caraway",Dairy and Egg Products,
8,"Calcium, Ca",Elements,mg,673.000,1008,"Cheese, caraway",Dairy and Egg Products,
9,"Iron, Fe",Elements,mg,0.640,1008,"Cheese, caraway",Dairy and Egg Products,


In [214]:
ndata.loc[30000]

nutrient                                       Glycine
nutgroup                                   Amino Acids
units                                                g
value                                             0.04
id                                                6158
food            Soup, tomato bisque, canned, condensed
fgroup                      Soups, Sauces, and Gravies
manufacturer                                          
Name: 30000, dtype: object

In [215]:
# 发现营养成分最为丰富的食物
by_nutrient = ndata.groupby(["nutgroup", "nutrient"])

get_maximum = lambda x: x.xs(x.value.idxmax())
get_minimum = lambda x: x.xs(x.value.idxmin())

max_foods = by_nutrient.apply(get_maximum)[["value", "food"]]
# 让food小一点
max_foods.food = max_foods.food.str[:50]

In [216]:
max_foods.loc["Amino Acids"]["food"]

nutrient
Alanine                           Gelatins, dry powder, unsweetened
Arginine                               Seeds, sesame flour, low-fat
Aspartic acid                                   Soy protein isolate
Cystine                Seeds, cottonseed flour, low fat (glandless)
Glutamic acid                                   Soy protein isolate
Glycine                           Gelatins, dry powder, unsweetened
Histidine                Whale, beluga, meat, dried (Alaska Native)
Hydroxyproline    KENTUCKY FRIED CHICKEN, Fried Chicken, ORIGINA...
Isoleucine        Soy protein isolate, PROTEIN TECHNOLOGIES INTE...
Leucine           Soy protein isolate, PROTEIN TECHNOLOGIES INTE...
Lysine            Seal, bearded (Oogruk), meat, dried (Alaska Na...
Methionine                    Fish, cod, Atlantic, dried and salted
Phenylalanine     Soy protein isolate, PROTEIN TECHNOLOGIES INTE...
Proline                           Gelatins, dry powder, unsweetened
Serine            Soy protein isolate, 